# Test GPU

In [1]:
import tensorflow as tf
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))


[[22. 28.]
 [49. 64.]]


# Imports and Data Directory

In [2]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import os
import pandas as pd
import numpy as np

data_dir = os.path.join('ECEN489Py4Data', 'data')

Using TensorFlow backend.


# Image Data Generation

In [3]:
#ImageDataGenerator implements functions useful for input image scaling and augmentation -- you may want more!

train_datagen = ImageDataGenerator(rescale=1./255)

valid_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)


In [4]:
train_generator = train_datagen.flow_from_directory(
        os.path.join(data_dir, 'train'),
        target_size=(32, 32),
        color_mode='rgb',
        batch_size=1,
        class_mode='categorical',
        shuffle=True,
        seed=1953)


valid_generator = valid_datagen.flow_from_directory(
        os.path.join(data_dir, 'validation'),
        target_size=(32, 32),
        color_mode='rgb',
        batch_size=1,
        class_mode='categorical',
        shuffle=False, #don't shuffle or label results will be all wrong
        seed=1953)


test_generator = test_datagen.flow_from_directory(
    os.path.join(data_dir, 'test'),
    target_size=(32, 32),
    color_mode='rgb',
    batch_size=1,
    class_mode='categorical',
    shuffle=False, #don't shuffle or label results will be all wrong
    seed=1953)


Found 2744 images belonging to 8 classes.
Found 929 images belonging to 8 classes.
Found 0 images belonging to 0 classes.


# CNN From Nowka

In [5]:
# neural network model
# you may want to vary these parameters, etc

num_classes = 8 # fixed by the number of classes of signs that we gave you. Dont change

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',input_shape = (32, 32, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'Adam', # may want to try others
              metrics = ['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
__________

In [7]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=4 # may need to increase if not seeing low enough losses
)


Instructions for updating:
Use tf.cast instead.
Epoch 1/4
2744/2744 [==============================] - 19s 7ms/step - loss: 0.4474 - acc: 0.8721 - val_loss: 0.1305 - val_acc: 0.9612
Epoch 2/4
2744/2744 [==============================] - 18s 6ms/step - loss: 0.1418 - acc: 0.9566 - val_loss: 0.1626 - val_acc: 0.9483
Epoch 3/4
2744/2744 [==============================] - 18s 6ms/step - loss: 0.1110 - acc: 0.9730 - val_loss: 0.1279 - val_acc: 0.9666
Epoch 4/4
2744/2744 [==============================] - 18s 6ms/step - loss: 0.0645 - acc: 0.9840 - val_loss: 0.0860 - val_acc: 0.9839


In [8]:
Y_pred = model.predict_generator(valid_generator, steps=STEP_SIZE_VALID)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(valid_generator.classes, y_pred))
print('Classification Report')
print(classification_report(valid_generator.classes, y_pred, target_names=valid_generator.class_indices))
print("F1 score (using average='micro')")
print(f1_score(y_true=valid_generator.classes, y_pred=y_pred, average='micro'))

Confusion Matrix
[[ 41  10   1   0   0   0   0   0]
 [  0  38  11   0   0   0   0   1]
 [  0   0 186  14   0   0   0   0]
 [  0   0   0 140  11   0   3   0]
 [  0   0   0   0   6  11   0   0]
 [  0   0   0   0   0  36  11   0]
 [  0   0   3   2   0   0 371  11]
 [ 11   0   0   0   0   0   2   9]]
Classification Report
              precision    recall  f1-score   support

       merge       0.79      0.79      0.79        52
       yield       0.79      0.76      0.78        50
   keepRight       0.93      0.93      0.93       200
speedLimit25       0.90      0.91      0.90       154
  pedestrian       0.35      0.35      0.35        17
speedLimit35       0.77      0.77      0.77        47
        stop       0.96      0.96      0.96       387
 signalAhead       0.43      0.41      0.42        22

   micro avg       0.89      0.89      0.89       929
   macro avg       0.74      0.73      0.74       929
weighted avg       0.89      0.89      0.89       929

F1 score (using average='micr

In [9]:
#USE CELLS BELOW TO COMPLETE THE EXERCISE WITH THE TEST SET

In [10]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
# Y_pred = model.predict_generator(test_generator, steps=STEP_SIZE_TEST)
# y_pred = np.argmax(Y_pred, axis=1)
# print('Confusion Matrix')
# print(confusion_matrix(test_generator.classes, y_pred))
# print('Classification Report')
# print(classification_report(test_generator.classes, y_pred, target_names=train_generator.class_indices))
# print("F1 score (using average='micro')")
# print(f1_score(y_true=test_generator.classes, y_pred=y_pred, average='micro'))

In [13]:
#This cell dumps out a file of which files were incorrectly predicted
#so you can see if you need more features, more training samples, etc
predicted_class_indices=np.argmax(Y_pred,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
# NOTE/TODO: Change this to test_generator later.
filenames=valid_generator.filenames
# filenames=test_generator.filenames
print(len(filenames))
print(len(predictions))
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)

929
929
